### Plasma retinol data regression (accuracy)

The Plasma retinol datais cited here: 

Nierenberg, D. W., Stukel, T. A., Baron, J. A., Dain, B. J., Greenberg, E. R., and Group,
S. C. P. S. (1989). Determinants of plasma levels of beta-carotene and retinol. American
Journal of Epidemiology, 130(3):511–521.

In [ ]:
%pip install contrastive-inverse-regression

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import contrastive_inverse_regression
from contrastive_inverse_regression import CIR

start_time = time.time()


data = pd.read_table('datasets/Retinol.txt',
                     header=None, delim_whitespace=True)
data = data.iloc[:, :]

# foreground data
fg = data.iloc[:, :12].dropna()

# foreground response (continuous)
Y = data.iloc[:, 12]
n, p = fg.shape  # foreground sample size
X = fg - np.mean(fg, axis=0)
X = X.values

# foreground slices
L = 10
partition = np.linspace(min(Y), max(Y), L)
labels = np.zeros(n)
for i in range(n):
    labels[i] = max(np.where(Y.iloc[i] >= partition)[0]) + 1

# background data
bg = data.iloc[:, :12].dropna()
m, p = bg.shape     # background sample size

# background data
Yt = data.iloc[:, 13]
Lt = 10
partitiont = np.linspace(min(Yt), max(Yt), Lt)
labelst = np.zeros(m)
for i in range(m):
    labelst[i] = max(np.where(Yt.iloc[i] >= partitiont)[0]) + 1

d = 2
alpha = 0.1

V_CIR = CIR(fg, labels, bg, labelst, alpha, d, continuous_Y=True)
X_CIR = X @ V_CIR


print("Training Linear Model")
model_CIR_LM = LinearRegression().fit(X_CIR, Y)
print("coefficients: ", model_CIR_LM.coef_)
print("intercept: ", model_CIR_LM.intercept_)
print("R-squared: ", model_CIR_LM.score(X_CIR, Y))

Y_pred = model_CIR_LM.predict(X_CIR)
MSE_CIR_LM = mean_squared_error(Y, Y_pred)

print("MSE: ", MSE_CIR_LM)


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken to run the code: {elapsed_time} seconds")

/Users/lianama/Library/Python/3.10/lib/python/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lianama/Library/Python/3.10/lib/python/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


---------------------------------------------------

Results for Scaled Gradient Projection Method 

---------------------------------------------------

   Obj. function = -1.945765e+05

   Gradient norm = 2.571787e+00 

   ||X^T*X-I||_F = 2.22e-16

   Iteration number = 3000

   Cpu time (secs) = 4.2504

   Number of evaluation(Obj. func) = 3899

Training Linear Model
coefficients:  [0.47531950807425  5.367195669470783]
intercept:  189.8920634920635
R-squared:  0.11696414685922552
This is MSE:  29478.367187909662
